# Model Evaluation

In [3]:
import os

In [1]:
%pwd

'/home/corti/Desktop/mlopsProject/research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'/home/corti/Desktop/mlopsProject'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path  # The root directory where the model and data are stored
    data_path: Path  # The path to the data for evaluation
    model_path: Path  # The path to the trained model
    tokenizer_path: Path  # The path to the tokenizer used in the model
    metric_file_name: Path  # The name of the file where the evaluation metrics will be stored

In [6]:
from ConversationSummarizer.constants import *
from ConversationSummarizer.utils.common import read_yaml, create_directories

In [8]:
# Define a class for managing configurations
class ConfigurationManager:
    # Initialize the ConfigurationManager with paths to the configuration and parameters files
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        # Read the configuration and parameters from the specified files
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Create the directory for storing artifacts
        create_directories([self.config.artifacts_root])

    # Method to get the configuration for the model evaluation
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        # Get the model evaluation configuration
        config = self.config.model_evaluation

        # Create the root directory for the model evaluation
        create_directories([config.root_dir])

        # Create a ModelEvaluationConfig object with the specified configuration
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            metric_file_name = config.metric_file_name
        )

        # Return the ModelEvaluationConfig object
        return model_evaluation_config


In [9]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_from_disk, load_metric
import torch
import pandas as pd
from tqdm import tqdm

/home/corti/anaconda3/envs/ConvSum/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-02-05 16:09:18,123: INFO: config: PyTorch version 2.2.0 available.]


In [10]:
class ModelEvaluation:
    # Initialize the ModelEvaluation with a configuration object
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    # Method to split a list into smaller batches
    def generate_batch_sized_chunks(self,list_of_elements, batch_size):
        """Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    # Method to calculate a metric on a test dataset
    def calculate_metric_on_test_ds(self,dataset, metric, model, tokenizer, 
                               batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu", 
                               column_text="article", 
                               column_summary="highlights"):
        # Split the articles and targets into batches
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        # For each batch of articles and targets
        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):
            
            # Tokenize the articles
            inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                            padding="max_length", return_tensors="pt")
            
            # Generate summaries with the model
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                            attention_mask=inputs["attention_mask"].to(device), 
                            length_penalty=0.8, num_beams=8, max_length=128)
            
            # Decode the generated summaries
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                    clean_up_tokenization_spaces=True) 
                for s in summaries]      
            
            # Replace the  token in the decoded summaries
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
            
            # Add the decoded summaries and the targets to the metric
            metric.add_batch(predictions=decoded_summaries, references=target_batch)
            
        # Compute and return the metric score
        score = metric.compute()
        return score

    # Method to evaluate the model
    def evaluate(self):
        # Determine the device to use
        device = "cuda" if torch.cuda.is_available() else "cpu"
        
        # Load the tokenizer and the model
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
       
        # Load the data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # Define the names of the ROUGE metrics to compute
        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
  
        # Load the ROUGE metric
        rouge_metric = load_metric('rouge')

        # Calculate the ROUGE scores on the test dataset
        score = self.calculate_metric_on_test_ds(
        dataset_samsum_pt['test'][0:10], rouge_metric, model_pegasus, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
            )

        # Create a dictionary of the ROUGE scores
        rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

        # Create a DataFrame of the ROUGE scores and save it to a CSV file
        df = pd.DataFrame(rouge_dict, index = ['pegasus'] )
        df.to_csv(self.config.metric_file_name, index=False)
        



In [12]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    raise e

[2024-02-05 16:11:30,196: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-05 16:11:30,200: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-05 16:11:30,202: INFO: common: created directory at: artifacts]
[2024-02-05 16:11:30,203: INFO: common: created directory at: artifacts/model_evaluation]


/tmp/ipykernel_319682/444777663.py:65: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')
/home/corti/anaconda3/envs/ConvSum/lib/python3.8/site-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
100%|██████████| 5/5 [02:08<00:00, 25.61s/it]

[2024-02-05 16:13:46,712: INFO: rouge_scorer: Using default tokenizer.]
